In [2]:
import random
import textwrap
import pandas as pd 

In [3]:
df = pd.read_json('hotpotqa-cte-inspect.jsonl', lines=True)
print(len(df))
df.head()

7405


,id,question,question_decomposition,context,answer,answers,predicted_triples,predicted_answer,score,n_hops,exact_match,f1,fuzzy_match
0,5a8b57f25542995d1e6f1371,Were Scott Derrickson and Ed Wood of the same ...,"[{'answer': '', 'id': 0, 'paragraph_support_id...",# Scott Derrickson\nScott Derrickson (born Jul...,yes,[yes],"Scott Derrickson, nationality, American\nEd Wo...",Yes,1.000000,2,1,1.000000,0
1,5a8c7595554299585d9e36b6,What government position was held by the woman...,"[{'answer': '', 'id': 0, 'paragraph_support_id...",# Shirley Temple\nShirley Temple Black (April ...,Chief of Protocol,[Chief of Protocol],"Shirley Temple, portrayed, Corliss Archer\nShi...",United States ambassador and Chief of Protocol...,0.461538,3,0,0.461538,0
2,5a85ea095542994775f606a8,"What science fantasy young adult series, told ...","[{'answer': '', 'id': 0, 'paragraph_support_id...",# The Hork-Bajir Chronicles\nThe Hork-Bajir Ch...,Animorphs,[Animorphs],"Animorphs, written by, K. A. Applegate\nThe Ho...",Animorphs,1.000000,5,1,1.000000,1
3,5adbf0a255429947ff17385a,Are the Laleli Mosque and Esma Sultan Mansion ...,"[{'answer': '', 'id': 0, 'paragraph_support_id...","# Laleli Mosque\nThe Laleli Mosque (Turkish: ""...",no,[no],"Laleli Mosque, located_in, Fatih\nEsma Sultan ...",No,1.000000,2,1,1.000000,0
4,5a8e3ea95542995a26add48d,"The director of the romantic comedy ""Big Stone...","[{'answer': '', 'id': 0, 'paragraph_support_id...",# Adriana Trigiani\nAdriana Trigiani is an Ita...,"Greenwich Village, New York City","[Greenwich Village, New York City]","Adriana Trigiani, based in, Greenwich Village\...",New York City,0.750000,2,0,0.750000,0


In [7]:
success_mask = df['f1'] > 0.7
fail_df = df[~success_mask]
success_df = df[success_mask]
print(len(fail_df), len(success_df), len(df))
print("{:.3f}".format(len(success_df)/len(df)))

1920 5485 7405
0.741


In [75]:
df.groupby('n_hops')[['exact_match', 'f1']].agg(['mean', 'count'])

exact_match              f1      
              mean count      mean count
n_hops                                  
2         0.666733  4990  0.815248  4990
3         0.643179  1774  0.810292  1774
4         0.698324   537  0.844756   537
5         0.775000    80  0.885774    80
6         0.642857    14  0.840476    14
7         0.444444     9  0.776720     9
8         1.000000     1  1.000000     1

In [121]:
def fixedwidth(text):
    return "\n".join(textwrap.wrap(text, width=80, replace_whitespace=False))

def format_row(row):
    output = []
    output.append(f"{row['id']} - {row['n_hops']} hops")
    output.append("="*80)
    output.append(fixedwidth(row['context']))
    output.append("="*80)
    output.append("Q: " + row['question'])
    output.append("Reference Answers: " + str(row['answers']))
    output.append("Predicted Answer: " + row['predicted_answer'])
    output.append("")
    output.append("# Predicted Triples")
    output.append(row['predicted_triples'])
    output.append("")
    output.append("# Scores")
    output.append("EM: {:.3f}".format(row['exact_match']))
    output.append("F1: {:.3f}".format(row['f1']))
    return "\n".join(output)

def present_row(row):
    print(format_row(row))

In [122]:
i = 0
row = df.iloc[i]
present_row(row)

5a8b57f25542995d1e6f1371 - 2 hops
# Scott Derrickson
Scott Derrickson (born July 16, 1966) is an American
director, screenwriter and producer.  He lives in Los Angeles, California.  He
is best known for directing horror films such as "Sinister", "The Exorcism of
Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic
Universe installment, "Doctor Strange."

# Ed Wood
Edward Davis Wood Jr.
(October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer,
producer, and director.
Q: Were Scott Derrickson and Ed Wood of the same nationality?
Reference Answers: ['yes']
Predicted Answer: Yes

# Predicted Triples
Scott Derrickson, nationality, American
Ed Wood, nationality, American

# Scores
EM: 1.000
F1: 1.000


In [123]:
i = 3
row = fail_df.iloc[i]
present_row(row)

5ae6050f55429929b0807a5e - 2 hops
# Jim Cummings
James Jonah Cummings (born November 3, 1952) is an American voice
actor and singer, who has appeared in almost 400 roles.  He is known for voicing
the title character from "Darkwing Duck", Dr. Robotnik from "Sonic the
Hedgehog", and Pete.  His other characters include Winnie the Pooh, Tigger, and
the Tasmanian Devil.  He has performed in numerous Disney and DreamWorks
animations including "Aladdin", "The Lion King", "Balto", "Antz", "The Road to
El Dorado", "Shrek", and "The Princess and the Frog".  He has also provided
voice-over work for video games, such as "Icewind Dale", "Fallout", "",
"Baldur's Gate", "Mass Effect 2", "", "", "", and "Splatterhouse".

# A Rather
Blustery Day
"A Rather Blustery Day" is a whimsical song from the Walt Disney
musical film featurette, "Winnie the Pooh and the Blustery Day".  It was written
by Robert & Richard Sherman and sung by Jim Cummings as "Pooh".
Q: This singer of A Rather Blustery Day also voiced

In [124]:
inspect_df = fail_df.copy().sort_values('f1')

In [125]:
from ipywidgets import widgets, HBox

def create_browse_app(df):
    def browse_failed(i=0):
        row = df.iloc[i]
        present_row(row)

    index = widgets.IntText(value=0, description='Index:')
    left_button = widgets.Button(description='Previous')
    right_button = widgets.Button(description='Next')

    def on_left_button_clicked(b):
        if index.value > 0:
            index.value -= 1

    def on_right_button_clicked(b):
        if index.value < len(df) - 1:
            index.value += 1

    left_button.on_click(on_left_button_clicked)
    right_button.on_click(on_right_button_clicked)

    ui = HBox([left_button, index, right_button])
    out = widgets.interactive_output(browse_failed, {'i': index})

    display(ui, out)

# Usage
create_browse_app(inspect_df)

Output()

In [127]:
df['triplets'] = df['predicted_triples'].map(str.splitlines)
df['n_triplets'] = df['triplets'].map(len)
df['n_triplets'].describe()

count    7405.000000
mean        2.627414
std         0.652369
min         1.000000
25%         2.000000
50%         3.000000
75%         3.000000
max        11.000000
Name: n_triplets, dtype: float64

In [128]:
poor_triplets_mask = df['n_triplets'] < df['n_hops']
df[poor_triplets_mask].describe()

,score,n_hops,exact_match,f1,fuzzy_match,n_triplets
count,1240.000000,1240.000000,1240.000000,1240.000000,1240.000000,1240.000000
mean,0.838866,3.584677,0.687097,0.838866,0.766129,2.330645
std,0.295809,0.751167,0.463862,0.295809,0.423462,0.546790
min,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000
25%,0.800000,3.000000,0.000000,0.800000,1.000000,2.000000
50%,1.000000,3.000000,1.000000,1.000000,1.000000,2.000000
75%,1.000000,4.000000,1.000000,1.000000,1.000000,3.000000
max,1.000000,8.000000,1.000000,1.000000,1.000000,5.000000


In [129]:
surprise_success_df = df[poor_triplets_mask&success_mask]
len(surprise_success_df)

957

In [131]:
create_browse_app(surprise_success_df)

Output()